# Art Docent 대화 디버깅

이 노트북에서 직접 대화를 시뮬레이션하며 post-curation 문제를 디버깅해보겠습니다.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

print("Environment loaded successfully!")
print(f"OpenAI API Key loaded: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")

: 

In [ ]:
# emotional_art_graph 모듈 임포트
from emotional_art_graph import graph, EmotionalArtState
from langchain_core.messages import HumanMessage, AIMessage

print("Graph imported successfully!")

In [ ]:
# 초기 상태 설정
initial_state = {
    "messages": [],
    "emotion_hints": [],
    "situation_hints": [],
    "conversation_depth": 0
}

# 설정
config = {
    "configurable": {
        "thread_id": "debug_thread",
        "user_id": "debug_user"
    }
}

print("Initial state and config ready!")

## 대화 시뮬레이션 함수

In [ ]:
async def send_message(user_input, current_state):
    """사용자 메시지를 보내고 응답을 받는 함수"""
    
    # 사용자 메시지 추가
    new_messages = current_state["messages"] + [HumanMessage(content=user_input)]
    new_state = {**current_state, "messages": new_messages}
    
    print(f"\n🧑 USER: {user_input}")
    print("="*50)
    
    # 그래프 실행
    result = await graph.ainvoke(new_state, config)
    
    # 결과 출력
    if result["messages"]:
        last_message = result["messages"][-1]
        if isinstance(last_message, AIMessage):
            print(f"🤖 AI: {last_message.content}")
    
    # 상태 정보 출력
    print("\n📊 STATE INFO:")
    print(f"  - Current Phase: {result.get('current_phase', 'None')}")
    print(f"  - Conversation Depth: {result.get('conversation_depth', 0)}")
    print(f"  - Consent for Reco: {result.get('consent_for_reco', 'None')}")
    print(f"  - Final Situation: {bool(result.get('final_situation'))}")
    print(f"  - Final Emotions: {bool(result.get('final_emotions'))}")
    print(f"  - Emotion Hints: {result.get('emotion_hints', [])}")
    print(f"  - Situation Hints: {result.get('situation_hints', [])}")
    
    return result

print("Message sending function ready!")

## 1단계: 인사 및 초기 대화

In [ ]:
# 첫 번째 메시지
state = await send_message("hello", initial_state)

In [ ]:
# 두 번째 메시지 - 감정 상태 공유
state = await send_message(
    "Hey… thanks for asking. It's been one of those long days, you know? Meetings, messages, and a dozen decisions waiting to be made. I keep telling myself this is what I wanted when I started my company, but lately, it's hard to feel that same spark.", 
    state
)

In [ ]:
# 세 번째 메시지 - 더 깊은 대화
state = await send_message(
    "I used to sketch a lot — nothing fancy, just pencil and paper. It helped me slow down and notice things. But lately, even when I try, my mind keeps drifting back to work. It's like I can't switch off the builder mode. Sometimes I go for a walk late at night, when the city's quiet. That helps a bit.", 
    state
)

In [ ]:
# 네 번째 메시지 - 더 많은 감정 공유
state = await send_message(
    "Yeah… I guess that's what I'm missing lately — that little escape. When I first started, building the company itself felt like art — every idea, every design decision was creative. But now it's mostly logistics, funding, and deadlines. It's hard to find that same sense of flow. Sometimes I wonder if I've turned something that used to heal me into something that drains me.", 
    state
)

## 2단계: 아트 추천 요청

In [ ]:
# 아트 추천 요청
state = await send_message("can you recommend me any artwork now?", state)

In [ ]:
# 아트 추천 동의
state = await send_message("yes please.", state)

In [ ]:
ㅁㄴㅇㄹ

## 3단계: 아트 큐레이션 후 - 문제 재현

In [ ]:
# 여기서 문제가 발생해야 함 - 아트 관련 질문인데 부적절한 응답이 나오는지 확인
print("\n" + "="*60)
print("🎯 CRITICAL TEST: Post-curation art question")
print("Expected: Helpful art guidance")
print("Bug: 'Totally get it!' generic response")
print("="*60)

state = await send_message("can you first teach me how to draw portraits?", state)

## 디버깅: Phase Detection 함수 직접 테스트

In [ ]:
# Phase Manager 직접 테스트
from emotional_art_graph import ConversationPhaseManager
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
phase_manager = ConversationPhaseManager(llm)

# 현재 상태로 phase detection 테스트
detected_phase = await phase_manager.detect_phase(state)
print(f"\n🔍 DETECTED PHASE: {detected_phase}")
print(f"Current phase in state: {state.get('current_phase', 'None')}")
print(f"\n📋 KEY STATE VALUES:")
print(f"  - final_situation: {state.get('final_situation', 'None')}")
print(f"  - final_emotions: {state.get('final_emotions', 'None')}")
print(f"  - consent_for_reco: {state.get('consent_for_reco', 'None')}")
print(f"  - current_phase: {state.get('current_phase', 'None')}")

## 추가 테스트: 다양한 메시지로 post-curation 상태 테스트

In [ ]:
# 다른 아트 관련 질문들 테스트
test_messages = [
    "What colors work best for beginners?",
    "How do I get started with watercolor painting?",
    "Tell me about perspective in drawing",
    "What art supplies should I buy?"
]

for i, msg in enumerate(test_messages, 1):
    print(f"\n\n🧪 TEST {i}: {msg}")
    print("="*60)
    test_state = await send_message(msg, state)
    
    # 각 테스트 후 phase detection 확인
    detected = await phase_manager.detect_phase(test_state)
    print(f"\n🔍 Phase detected for this test: {detected}")
    
    await asyncio.sleep(0.5)  # 요청 간격

## 상태 분석

In [ ]:
# 최종 상태 상세 분석
print("\n" + "="*60)
print("📋 FINAL STATE ANALYSIS")
print("="*60)

for key, value in state.items():
    if key == "messages":
        print(f"\n📨 Messages ({len(value)} total):")
        for i, msg in enumerate(value[-5:], 1):  # 마지막 5개만 표시
            msg_type = "🧑 Human" if isinstance(msg, HumanMessage) else "🤖 AI"
            content = msg.content[:100] + "..." if len(str(msg.content)) > 100 else msg.content
            print(f"  {i}. {msg_type}: {content}")
    else:
        print(f"\n{key}: {value}")